In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials

<ipython-input-1-717e3dedb123>:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#for single link testing #link = "https://finance.yahoo.com/quote/HLAG.DE/key-statistics?p=HLAG.DE"

link_list = ["https://finance.yahoo.com/quote/HLAG.DE/key-statistics?p=HLAG.DE", "https://finance.yahoo.com/quote/AAD.DE/key-statistics?p=AAD.DE", "https://finance.yahoo.com/quote/AOF.F/key-statistics?p=AOF.F" ]
headers_http = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}

In [3]:
"""
This code was used, to extract the Table Headers, which i fill in here hardcoded, because its the same financial Names in every Stock
finance_names = []

#noticed only 50 of 60 headlines were found - 10 have other class name (for no reason) catched them with RegEX -- ("td", class_="Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pend(10px)")
class_pattern = re.compile(r"(^|\s)Pos\(st\) Start\(0\) Bgc\(\$lv2BgColor\) fi-row:h_Bgc\(\$hoverBgColor\) Pend\(10px\)(\s|$)")

column_names = soup.find_all("td", class_=class_pattern)
for element in column_names:
    finance_names.append(element.text)
for idx, element in enumerate(finance_names):
    finance_names[idx] =  element.replace(" ","_")
print(finance_names) """





'\nThis code was used, to extract the Table Headers, which i fill in here hardcoded, because its the same financial Names in every Stock\nfinance_names = []\n\n#noticed only 50 of 60 headlines were found - 10 have other class name (for no reason) catched them with RegEX -- ("td", class_="Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pend(10px)")\nclass_pattern = re.compile(r"(^|\\s)Pos\\(st\\) Start\\(0\\) Bgc\\(\\$lv2BgColor\\) fi-row:h_Bgc\\(\\$hoverBgColor\\) Pend\\(10px\\)(\\s|$)")\n\ncolumn_names = soup.find_all("td", class_=class_pattern)\nfor element in column_names:\n    finance_names.append(element.text)\nfor idx, element in enumerate(finance_names):\n    finance_names[idx] =  element.replace(" ","_")\nprint(finance_names) '

In [4]:
headers_column = ['Market_Cap_(intraday)_', 'Enterprise_Value_', 'Trailing_P/E_', 'Forward_P/E_', 'PEG_Ratio_(5_yr_expected)_', 'Price/Sales_(ttm)', 
                  'Price/Book_(mrq)', 'Enterprise_Value/Revenue_', 'Enterprise_Value/EBITDA_', 'Beta_(5Y_Monthly)_', '52-Week_Change_3', 
                  'S&P500_52-Week_Change_3', '52_Week_High_3', '52_Week_Low_3', '50-Day_Moving_Average_3', '200-Day_Moving_Average_3', 'Avg_Vol_(3_month)_3', 
                  'Avg_Vol_(10_day)_3', 'Shares_Outstanding_5', 'Implied_Shares_Outstanding_6', 'Float_8', '%_Held_by_Insiders_1', '%_Held_by_Institutions_1', 
                  'Shares_Short__4', 'Short_Ratio__4', 'Short_%_of_Float__4', 'Short_%_of_Shares_Outstanding__4', 'Shares_Short_(prior_month_)_4', 
                  'Forward_Annual_Dividend_Rate_4', 'Forward_Annual_Dividend_Yield_4', 'Trailing_Annual_Dividend_Rate_3', 'Trailing_Annual_Dividend_Yield_3', 
                  '5_Year_Average_Dividend_Yield_4', 'Payout_Ratio_4', 'Dividend_Date_3', 'Ex-Dividend_Date_4', 'Last_Split_Factor_2', 'Last_Split_Date_3', 
                  'Fiscal_Year_Ends_', 'Most_Recent_Quarter_(mrq)', 'Profit_Margin_', 'Operating_Margin_(ttm)', 'Return_on_Assets_(ttm)', 'Return_on_Equity_(ttm)', 
                  'Revenue_(ttm)', 'Revenue_Per_Share_(ttm)', 'Quarterly_Revenue_Growth_(yoy)', 'Gross_Profit_(ttm)', 'EBITDA_', 'Net_Income_Avi_to_Common_(ttm)', 
                  'Diluted_EPS_(ttm)', 'Quarterly_Earnings_Growth_(yoy)', 'Total_Cash_(mrq)', 'Total_Cash_Per_Share_(mrq)', 'Total_Debt_(mrq)', 'Total_Debt/Equity_(mrq)', 
                  'Current_Ratio_(mrq)', 'Book_Value_Per_Share_(mrq)', 'Operating_Cash_Flow_(ttm)', 'Levered_Free_Cash_Flow_(ttm)']

data_list = []
finance_numbers = []
name_list = []
link_list1 = []


In [5]:
for link in link_list:
    r = requests.get(link, headers=headers_http)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, "html.parser")
        
        finanz_zahlen = soup.find_all("td", class_="Fw(500) Ta(end) Pstart(10px) Miw(60px)", string=True)
        firmenname = soup.find_all("h1", class_="D(ib) Fz(18px)", string=True)
        finance_numbers = []  # Initialize finance_numbers within the loop
        

        for element in finanz_zahlen:
            finance_numbers.append(element.text)

        for idx, element in enumerate(finance_numbers):
            if element.endswith("M"):
                finance_numbers[idx] = float(element[:-1]) * 1e6
            elif element.endswith("B"):
                finance_numbers[idx] = float(element[:-1]) * 1e9
            elif element.endswith("k"):
                finance_numbers[idx] = float(element[:-1]) * 1e3
            elif element.endswith("%"):
                finance_numbers[idx] = element

        data_list.extend(finance_numbers)  # Append finance_numbers to data_list after processing
        name_list.extend(firmenname)  # Append Company name to names list after processing

    else:
        print("Seems like there was an Error")

# Converting Finance Numbers into a Dataframe and Reshape them into 60 columns (this sorts them as well)
data_array = np.array(data_list)
data_array = data_array.reshape(len(link_list), 60)
df_finance = pd.DataFrame(data_array, columns=headers_column)  

# do the same for the Company name
name_array = np.array(name_list)
name_array = name_array.reshape(len(link_list), 1)
df_names = pd.DataFrame(name_array, columns=["Company Name"])

# Get 1 Column for the Links, to look up further Data if Stock has good measurements

df_links = pd.DataFrame(link_list, columns=["Link_Yahoo_Finance"])

# create final DF to write into Google Sheets later
df_final = pd.concat([df_names,df_links, df_finance], axis=1)

# just a quick test if whole list was successfully crawled
print("Script Run Successfully!")

Script Run Successfully!


In [6]:

link_list1

[]

In [7]:
df_final.head()

,Company Name,Link_Yahoo_Finance,Market_Cap_(intraday)_,Enterprise_Value_,Trailing_P/E_,Forward_P/E_,PEG_Ratio_(5_yr_expected)_,Price/Sales_(ttm),Price/Book_(mrq),Enterprise_Value/Revenue_,...,Diluted_EPS_(ttm),Quarterly_Earnings_Growth_(yoy),Total_Cash_(mrq),Total_Cash_Per_Share_(mrq),Total_Debt_(mrq),Total_Debt/Equity_(mrq),Current_Ratio_(mrq),Book_Value_Per_Share_(mrq),Operating_Cash_Flow_(ttm),Levered_Free_Cash_Flow_(ttm)
0,Hapag-Lloyd Aktiengesellschaft (HLAG.DE),https://finance.yahoo.com/quote/HLAG.DE/key-st...,23870000000.0,20820000000.0,3.70,N/A,N/A,1.08,1.20,0.95,...,36.52,-95.00%,6360000000.0,36.17,5460000000.0,27.57%,1.82,112.41,9190000000.0,3970000000.0
1,Amadeus FiRe AG (AAD.DE),https://finance.yahoo.com/quote/AAD.DE/key-sta...,594280000.0,643170000.0,14.76,N/A,N/A,1.44,4.22,1.49,...,7.41,6.20%,19570000.0,3.42,108330000.0,75.62%,0.61,24.63,90850000.0,77550000.0
2,ATOSS Software AG (AOF.F),https://finance.yahoo.com/quote/AOF.F/key-stat...,1960000000.0,1920000000.0,68.75,N/A,N/A,14.08,45.25,14.35,...,3.43,57.60%,56090000.0,7.05,8660000.0,20.89%,N/A,5.23,N/A,N/A


In [8]:
# Define the scope
scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
path = "evident-catcher-416200-7712669a9428.json"
# Authenticate using credentials
credentials = ServiceAccountCredentials.from_json_keyfile_name(path, scope)
client = gspread.authorize(credentials)

# Open the Google Sheet (replace 'MySheet' with your sheet name)
sheet = client.open('SDAX_Data_Dump').sheet1


# Write DataFrame to Google Sheet
# First, clear the existing contents of the sheet
sheet.clear()
# Then, write the DataFrame to the sheet
sheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

{'spreadsheetId': '19yWEwyOEKP_sS_w2laKwD7uIcrST97LpABUaXEw8tQQ',
 'updatedRange': 'Sheet1!A1:BJ4',
 'updatedRows': 4,
 'updatedColumns': 62,
 'updatedCells': 248}